In [1]:
from libraries import *

In [2]:
df = pd.read_csv("Title18.csv")
df

,Section,Url,Content,Metadata
0,103. âFront Matter,https://uscode.house.gov/view.xhtml?req=granul...,18 USC Ch. 103: Front Matter Result...,"Amendments1992-Pub. L. 102â519, title I, Â§1..."
1,2111. Special maritime and territorial jurisdi...,https://uscode.house.gov/view.xhtml?req=granul...,"Whoever, within the special maritime and terri...","Amendments1994-Pub. L. 103â322 inserted ""or ..."
2,2112. Personal property of United States,https://uscode.house.gov/view.xhtml?req=granul...,Whoever robs or attempts to rob another of any...,"Amendments1994-Pub. L. 103â322 inserted ""or ..."
3,2113. Bank robbery and incidental crimes,https://uscode.house.gov/view.xhtml?req=granul...,"(a) Whoever, by force and violence, or by inti...",References in TextSection 1(b) of the Internat...
4,"2114. Mail, money, or other property of United...",https://uscode.house.gov/view.xhtml?req=granul...,(a) Assault.-A person who assaults any person ...,Amendments1996-Pub. L. 104â294 amended Pub. ...
...,...,...,...,...
1642,50. Prompt Disposition,https://uscode.house.gov/view.xhtml?req=granul...,Scheduling preference must be given to crimina...,NaN
1643,51. Preserving Claimed Error,https://uscode.house.gov/view.xhtml?req=granul...,(a) Exceptions Unnecessary. Exceptions to ruli...,NaN
1644,52. Harmless and Plain Error,https://uscode.house.gov/view.xhtml?req=granul...,"(a) Harmless Error. Any error, defect, irregul...",NaN
1645,53. Courtroom Photographing and Broadcasting P...,https://uscode.house.gov/view.xhtml?req=granul...,Except as otherwise provided by a statute or t...,NaN


In [3]:
df['Content'] = df['Content'].fillna("").astype(str)

In [3]:
max = 0
for i in range(len(df['Content'])):
    try:
        if len(df['Content'][i]) > max:
            max = len(df['Content'][i])
    except:
        print(i)        
max

121468

In [4]:
print(len(df))

1647


In [5]:
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased") # nlpaueb/legal-bert-base-uncased   |  
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Step 3: Function to get embeddings
def get_embedding(text, tokenizer, model):
    # Tokenize and encode the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    
    # Compute embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the mean of the token embeddings (for sentence embedding)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

# Step 4: Encode the entire 'Content' column
embeddings = np.array([get_embedding(text, tokenizer, model).cpu().numpy() for text in df['Content']]).astype('float32')

# # Step 5: Initialize a FAISS index with L2 (Euclidean) distance
# dimension = embeddings.shape[1]  # Get the dimensionality of the embeddings
# index = faiss.IndexFlatL2(dimension)  # IndexFlatL2 is a flat index (no compression)

# # Step 6: Add the vectors to the index
# index.add(embeddings)

# # Optional: Verify the number of vectors in the index
# print(f"Number of vectors in the index: {index.ntotal}")


In [6]:
embeddings.shape

(1647, 768)

In [7]:
np.savetxt("embeddings-complete-legal-bert-base-uncased.csv", embeddings, delimiter=',')

In [12]:
# test = ''' 
#  for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''

# # Step 8: Encode the test query
# test_embedding = get_embedding(test, tokenizer, model).reshape(1, -1).cpu().numpy().astype('float32')

# # Step 9: Example: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 10: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])

: 

Cosine Similarity for FIASS

In [ ]:
# import faiss
# import numpy as np

# # Assuming 'embeddings' contains the embeddings loaded from your CSV file
# # Step 1: Normalize the embeddings for cosine similarity
# def normalize(embeddings):
#     norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
#     return embeddings / norms

# # Step 2: Normalize the existing embeddings
# normalized_embeddings = normalize(embeddings)

# # Step 3: Initialize a FAISS index with Inner Product (for cosine similarity)
# dimension = normalized_embeddings.shape[1]
# index = faiss.IndexFlatIP(dimension)  # IndexFlatIP is for inner product (dot product)

# # Step 4: Add the normalized vectors to the index
# index.add(normalized_embeddings)

# # Optional: Verify the number of vectors in the index
# print(f"Number of vectors in the index: {index.ntotal}")

# # Step 5: Normalize the test query embedding
# test = ''' 
# for which the maximum fine is no greater than the amount set forth
# for such an offense in section 3571(b)(6) or (7) in the case of an individual or section 3571(c)(6) or
# (7) in the case of an organization.
# '''
# # Get the test embedding using the existing model and tokenizer
# test_embedding = get_embedding(test, tokenizer, model).cpu().numpy().astype('float32')
# test_embedding = normalize(test_embedding.reshape(1, -1))

# # Step 6: Search for the nearest neighbors
# D, I = index.search(test_embedding, k=2)  # Search for 2 nearest neighbors

# # Step 7: Print the distances and indices of the nearest neighbors
# print("Distances:", D)
# print("Indices:", I)

# # Optionally print the neighbor texts
# for idx in I[0]:
#     print("Neighbor text:", df['Content'].iloc[idx])
